In [1]:
import numpy as np
import pandas as pd
from numpy.random import rand
import gzip
import json
import ast
from scipy.cluster.vq import kmeans2
import matplotlib.pyplot as plt

In [5]:
# we test with bin matrices which has 300 columns corresponding to dimension of points
# we take 10 rows which is the number of accounts, this value should be around 10^6 to be set later
mu = 0
sigma = 1.0
A = np.random.normal(mu, sigma, (10,300))
#Now do the 2-means and get the output.
centroid, label = kmeans2(A, 2, minit='points')
#centroid will be two arrays each of 10 dimension for the two clusters
# label[i] is the code or index of the centroid the ith observation is closest to.
label[2],label[4],label[7]# sample testing

(1, 1, 0)

In [7]:
# Now we write the function that partitons A into A0 and A1
# A0 and A1 are lists which will contain only the indices of the rows which are supposed to be in A0 and A1
# partition takes an arr which means it contains the matrix made up of A[arr[i]]
# we need to form matrix from the rows of A as mentioned in arr
arr = list(range(10))# later this should be list(range(10**6))
def partition(arr):
    matrix = []
    for i in range(len(arr)):
        matrix.append(A[i,:])
    matrix = np.array(matrix)
    centroid, label = kmeans2(matrix, 2, minit='points')
    A0,A1 = [],[]
    for i in range(len(arr)):# this is the num of the rows, i.e. the num of accts
        if label[i] == 0:
            A0.append(i)# means A[i] should be in this as a row
        else:
            A1.append(i)# means A[i] should be in this as a row
    #A0 = np.array(A0)
    #A1 = np.array(A1)
    return (A0,A1)# returns two lists so that x = A0[i], A[x] will give the reqd row

x,y = partition(A)
x

[1, 2, 7]

In [17]:
tau = 3
# data contains lists which contain indices i. A[i] will be the reqd rows, to start with data = [0,...,9]
class Node:
    def __init__ (self, data, parent = None):
        #self.width = width
        #self.height = height
        self.parent = parent
        self.children = []
        self.data = data

    @property
    def level (self):
        return 0 if not self.parent else 1 + self.parent.level

    def split (self):
        #if self.children == []: return
        numrow = len(self.data)
        if numrow <= tau:
            self.children = []
            return
        A1,A2 = partition(self.data)
        self.children = [Node(A1, self),Node(A2, self)]
        
    def splitUntilLevel (self, maxLevel):
        if maxLevel <= self.level: return
        self.split ()
        for child in self.children: child.splitUntilLevel (maxLevel)

    #def __str__ (self):# this needs to be written suiably for printing the tree
     #   s = "{}".format(self.data)
      #  for child in self.children: s += str (child)
       # return s
    def sizeofNodes(self):
        s = [len(self.data)]
        for child in self.children:
            s.append(child.sizeofNodes())
        #print(s)
        return s
    
N = Node(list(range(10)))
N.splitUntilLevel (3)
print(N.sizeofNodes())


[10, [7, [2], [5, [1], [4]]], [3]]
